In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
import scipy as sp
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from fractions import Fraction
from decimal import Decimal
from IPython.core.display import HTML


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh_min = 64
refRatio = 2

diff = 'CD'
order = 6
RL = 'R'

L = 1.
locs = [0.5]
epsilons = [1, 0.5]# 11.68]
mus = [1, 0.5]#0.99837]

In [3]:
omegaAMR = BT.Grid(nh_min)
finehalf = list(int(nh_min / 2) + np.arange(-4, 4)) # list(np.arange(int(nh_min / refRatio)))# + int(nh_min / refRatio)) # list(np.arange(int(nh_min / 4), int(3 * nh_min / 4)))
omegaAMR.AddPatch(refRatio, finehalf)
degFreed = omegaAMR.degFreed
nh_max = omegaAMR.nh_max

omegaF = BT.Grid(nh_max)
omegaC = BT.Grid(nh_min)

wavesAMR = WT.MakeWaves(omegaAMR)
nullspace = OT.FindNullspace(omegaAMR, wavesAMR)
nullspaceBlock = OT.Block(nullspace, var = 2)
restrictOp = GTT.CoarsenOp(omegaAMR)
restrictOpBlock = OT.Block(restrictOp, var = 2)

wavesF = WT.MakeWaves(omegaF)
wavesC = WT.MakeWaves(omegaC)
print(finehalf)

[28, 29, 30, 31, 32, 33, 34, 35]


In [4]:
physicsAMR = BT.PhysProps(omegaAMR, epsilons, mus, locs, L)
c = physicsAMR.cVec
cMatAMR = physicsAMR.cMat

physicsC = BT.PhysProps(omegaC, epsilons, mus, locs, L)
cC = physicsC.cVec
cMatC = physicsC.cMat

physicsF = BT.PhysProps(omegaF, epsilons, mus, locs, L)
cF = physicsF.cVec
cMatF = physicsF.cMat

cVecC = physicsC.cVec
c1 = cVecC[0]
c2 = cVecC[-1]

In [5]:
def CDFace(orderIn):
    errorLoc = 'ERROR:\nOperatorTools:\nCDFace:\n'
    errorMess = ''
    if (orderIn % 2 == 0):
        order = orderIn
    else:
        order = int(orderIn + 1)
    
    stenc = np.zeros(order + 1)
    
    if (order == 2):
        face = (1. / 2.) * np.asarray([1, 1])
    else:
        if (order == 4):
            face = (1. / 12.) * np.asarray([-1, 7, 7, -1])
        else:
            if (order == 6):
                face = (1. / 60.) * np.asarray([1, -8, 37, 37, -8, 1])
            else:
                errorMess = 'This program is not designed to handle this order of accuracy for center-difference operators.'
    
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    return face

In [6]:
def UDFace(orderIn):
    errorLoc = 'ERROR:\nOperatorTools:\nUDFace:\n'
    errorMess = ''
    if (orderIn % 2 == 0):
        order = int(orderIn + 1)
    else:
        order = orderIn
    
    stenc = np.zeros(order + 1)
    
    if (order == 1):
        face = np.asarray([1])
    else:
        if (order == 3):
            face = (1. / 6.) * np.asarray([-1, 5, 2])
        else:
            if (order == 5):
                face = (1. / 60.) * np.asarray([2, -13, 47, 27, -3])
            else:
                if (order == 7):
                    face = (1. / 420.) * np.asarray([-3, 25, -101, 319, 214, -38, 4])
                else:
                    if (order == 9):
                        face = (1. / 2520.) * np.asarray([4, -41, 199, -641, 1879, 1375, -305, 55, -5])
                    else:
                        errorMess = 'This program is not designed to handle this order of accuracy for forward- and backward-difference operators.'
    
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    return face

In [7]:
def DDFace(order):
    stenc = UDFace(order)[::-1]
    return stenc

In [8]:
def FaceOp(omega, order, diff, RL, Ng, otherFace = False):
    errorLoc = 'ERROR:\nOperatorTools:\nFaceOp:\n'
    errorMess = ''
    
    if (diff == 'C' or diff == 'CD'):
        stenc = CDFace(order)
        if (order % 2 == 0):
            orderStenc = int(order - 1) # order
        else:
            orderStenc = order # int(order + 1)
        off = int(orderStenc / 2)
        hiBound = (off + 1.) / 2. # off / 2.
        if (RL == 'L'):
            off = int(off + 1)
            hiBound = (off - 1.) / 2. # off / 2.
        else:
            if (RL != 'R'):
                errorMess = 'Variable RL must be set either to \'R\' for right-moving wave or \'L\' for left-moving wave.'
        loBound = -off / 2.
        
        print('loBound:', loBound)
        print('hiBound:', hiBound)
    else:
        orderStenc = order
        if (order % 2 == 0):
            orderStenc = order # int(order + 1)
        else:
            orderStenc = int(order - 1) # order
        off = int(orderStenc / 2) # int((orderStenc + 1) / 2)
        if (diff == 'U' or diff == 'UD'):
            loBound = -off / 2.
            hiBound = off / 2. #(off - 1.) / 2.
            stenc = UDFace(order)
            if (RL == 'L'):
                stenc = stenc[::-1]
            else:
                if (RL != 'R'):
                    errorMess = 'Variable RL must be set either to \'R\' for right-moving wave or \'L\' for left-moving wave.'
            
#         else:
#             if (diff == 'D' or diff == 'DD'):
#                 stenc = DDFace(order) # OT.DDStencil(order)
#                 # off = int(off - 1)
#                 loBound = -off / 2. # -(off + 1.) / 2. # -off / 2.
#                 hiBound = off / 2. # (off + 1.) / 2.
        else:
            errorMess = 'Invalid entry for variable diff. Must be \'C\' or \'CD\', for center-difference, or \'U\' or \'UD\', for upwind-difference.'
    if (Ng > off + 1):
        errorMess = 'Too many ghost cells for this order of face approximation!'
    
    val = Ng + off
    
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
#     stenc = np.ones(orderStenc + 1)
    
    np.set_printoptions(precision=24, suppress=True)
    
    degFreed = omega.degFreed
    hs = omega.h
    
    halfDeg = int(degFreed / 2)
      
    spots = np.roll(hs, -1) - hs
    
    if (all(spots == 0)):
        p = []
        q = []
        NU = False
    else:
        # Index before fine-coarse interface
        p = np.where(spots > 0)[0][0]
        # Index before coarse-fine interface
        q = np.where(spots < 0)[0][0]
        NU = True
    
    
    
    if (otherFace):
        val = val - 1
        if (RL == 'R'): # Then everything shifts left.
            off = off + 1
            loBound = loBound - 0.5
            hiBound = hiBound - 0.5
#             if (NU):
#                 p = (p - 1) % (degFreed + 2 * Ng)
#                 q = (q - 1) % (degFreed + 2 * Ng)
        else:
            off = off - 1
            loBound = loBound + 0.5
            hiBound = hiBound + 0.5
#             if (NU):
#                 p = (p + 1) % (degFreed + 2 * Ng)
#                 q = (q + 1) % (degFreed + 2 * Ng)
    
    print('off is', off)
    print('val is', val)
    
    cellFaces = np.linspace(loBound, hiBound, num = orderStenc + 1)
    print('cellFaces before:')
    print(cellFaces)
    print(np.where(cellFaces == 0)[0])
    zeroLoc = np.where(cellFaces == 0)[0]
    if (len(zeroLoc) != 0):
        cellFaces = np.delete(cellFaces, zeroLoc[0])
        
    print('cellFaces after:')
    print(cellFaces)
    polys = np.shape(cellFaces)[0]
    polyStencSet = [[] for i in range(polys)]
    
    addZeros = np.zeros((polys, Ng), float)

    for i in range(polys):
        polyStencSet[i], n_c, n_f = GTT.CentGhost(omega, order, cellFaces[i])
    
    polyStencSet = np.asarray(polyStencSet)
    if ((Ng != 0) and (polys != 0)):
        polyStencSet = np.hstack([addZeros, polyStencSet, addZeros])

    print('Shape of polyStencSet:')
    print(np.shape(polyStencSet))
    
    IMat = np.eye(degFreed + 2 * Ng, degFreed + 2 * Ng) # np.eye(degFreed, degFreed)
    
    # YOU'RE GONNA NEED THESE TO RESTRICT FOR HIGHER EVEN ORDERS, TOO.
    
    
    polyMatU = IMat + 0
    
    
    mat = np.zeros((degFreed, degFreed + 2 * Ng), float) # np.zeros((degFreed, degFreed), float)
    faceOp = mat + 0
    
    for d in range(orderStenc + 1):
        s = int(off - d)
        
        print('s:', s)
        
        derivMat = mat + 0
        np.fill_diagonal(derivMat[:, Ng:Ng+degFreed], stenc[d]) # np.fill_diagonal(derivMat, stenc[d])
        print('BEFORE:')
        print(derivMat)
        derivMat = np.roll(derivMat, -s, axis = 1) # np.roll(derivMat, s, axis = 0)
        print('AFTER:')
        print(derivMat)
        
        polyMat = IMat + 0
        
        if (NU):
            if (s > 0):
                j = int(off - s)
                pAt = (p + Ng) % (degFreed + 2 * Ng) # p
                pLo = (p + Ng - 1) % (degFreed + 2 * Ng) # (p - 1) % degFreed
                qAt = (q - s + Ng + 1) % (degFreed + 2 * Ng) # (q - s + 1) % degFreed #(q + 1) % degFreed
                for i in range (s):
                    polyMat[pAt, :] = 0
                    polyMat[pAt, pLo:pLo+2] = 0.5
                    polyMat[qAt, :] = polyStencSet[j, :]
                    pAt = (pAt - 1) % (degFreed + 2 * Ng) # (pAt - 1) % degFreed
                    pLo = (pLo - 2) % (degFreed + 2 * Ng) # (pLo - 2) % degFreed
                    qAt = (qAt + 1) % (degFreed + 2 * Ng) # (qAt + 1) % degFreed
                    j = int(j + 1)

            if (s < 0):
                j = int(off) # - s - 1
                qAt = (q + Ng + 1) % (degFreed + 2 * Ng) # (q + 1) % degFreed
                qLo = (q + Ng + 1) % (degFreed + 2 * Ng) # (q + 1) % degFreed
                pAt = (p + Ng + 1) % (degFreed + 2 * Ng) # (p + 1) % degFreed
                for i in range(abs(s)):
                    polyMat[qAt, :] = 0
                    polyMat[qAt, qLo:qLo+2] = 0.5
                    polyMat[pAt, :] = polyStencSet[j, :]
                    qAt = (qAt + 1) % (degFreed + 2 * Ng) # (qAt + 1) % degFreed
                    qLo = (qLo + 2) % (degFreed + 2 * Ng) # (qLo + 2) % degFreed
                    pAt = (pAt + 1) % (degFreed + 2 * Ng) # (pAt + 1) % degFreed
                    j = int(j + 1) # - 1
        
        matThis = derivMat @ polyMat
        
        faceOp = faceOp + matThis
    
    finRow = np.zeros((1, halfDeg + 2 * Ng), float)
    finRowMaj = np.zeros((1, degFreed + 2 * Ng), float)
    
    
#     rowSums = np.round(sum(faceOp, axis = 1), 11)
#     probRows = np.where(rowSums != 0)[0]
    
#     for row in probRows:
#         for j in range(degFreed):
#             frac = Fraction(faceOp[row, j]).limit_denominator(10**order)
#             num = frac.numerator
#             denom = frac.denominator
#             faceOp[row, j] = num / denom
    
    faceOp1 = faceOp[:halfDeg, :halfDeg + 2 * Ng]
    faceOp2 = faceOp[-halfDeg:, -halfDeg - 2 * Ng:]
    
    if (RL == 'R'):
        if ((order > 1) and (val > 0)):
            finRow[0, :val] = stenc[-val:]
            finRowMaj[0, :val] = stenc[-val:]
        faceOp1 = np.concatenate((finRow, faceOp1), axis = 0)
        faceOp2 = np.concatenate((finRow, faceOp2), axis = 0)
        faceOp = np.concatenate((finRowMaj, faceOp), axis = 0)
    else:
        if ((order > 1) and (val > 0)):
            finRow[0, -val:] = stenc[:val]
            finRowMaj[0, -val:] = stenc[:val]
        faceOp1 = np.concatenate((faceOp1, finRow), axis = 0)
        faceOp2 = np.concatenate((faceOp2, finRow), axis = 0)
        faceOp = np.concatenate((faceOp, finRowMaj), axis = 0)
    
    print('stenc is')
    print(stenc)
    print('finRowMaj is')
    print(finRowMaj)
    
    # hMat = OT.StepMatrix(omega)
    
    # derivOp = hMat @ derivOp
        
    return faceOp1, faceOp2, faceOp


In [9]:
if (order == 1):
    Ng = 0
else:
    Ng = order - 2
Ng = 0
faceOp1, faceOp2, faceOp = FaceOp(omegaAMR, order, diff, RL, Ng)
print('D =')
print(faceOp)
print('')

loBound: -1.0
hiBound: 1.5
off is 2
val is 2
cellFaces before:
[-1.  -0.5  0.   0.5  1.   1.5]
[2]
cellFaces after:
[-1.  -0.5  0.5  1.   1.5]
Shape of polyStencSet:
(5, 72)
s: 2
BEFORE:
[[0.016666666666666666 0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.           

[[0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.         

[[0.6166666666666667 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.        

[[0.6166666666666667 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.        

[[0.                 0.6166666666666667 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.        

[[0.016666666666666666 0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.         

[[-0.13333333333333333     0.016666666666666666    0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.           

In [10]:
faceOp1, faceOp2, faceOpOther = FaceOp(omegaAMR, order, diff, RL, Ng, True)
print('D =')
print(faceOpOther)

loBound: -1.0
hiBound: 1.5
off is 3
val is 1
cellFaces before:
[-1.5 -1.  -0.5  0.   0.5  1. ]
[3]
cellFaces after:
[-1.5 -1.  -0.5  0.5  1. ]
Shape of polyStencSet:
(5, 72)
s: 3
BEFORE:
[[0.016666666666666666 0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.           

[[-0.13333333333333333  0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.        

[[ 0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.        

[[0.6166666666666667 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.        

[[0.6166666666666667 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.        

[[0.016666666666666666 0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.                   0.         

[[ 0.016666666666666666    0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.           

In [11]:
def SpaceDeriv(omega, order, diff):
    errorLoc = 'ERROR:\nOperatorTools:\nMakeSpaceDeriv:\n'
    errorMess = ''
    if (diff == 'C' or diff == 'CD'):
        stenc = OT.CDStencil(order)
        if (order % 2 == 0):
            orderStenc = order
        else:
            orderStenc = int(order + 1)
        off = int(orderStenc / 2)
        loBound = -off / 2.
        hiBound = off / 2.
    else:
        orderStenc = order
        if (order % 2 == 0):
            orderStenc = int(order + 1)
        else:
            orderStenc = order
        off = ((orderStenc + 1) / 2)
        if (diff == 'U' or diff == 'UD'):
            stenc = OT.UDStencil(order)
            loBound = -off / 2.
            hiBound = (off - 1.) / 2.
        else:
            if (diff == 'D' or diff == 'DD'):
                stenc = OT.DDStencil(order)
                off = int(off - 1)
                loBound = -off / 2.
                hiBound = (off + 1.) / 2.
            else:
                errorMess = 'Invalid entry for variable diff. Must be \'C\', \'U\', \'D\', \'CD\', \'UD\', or \'DD\'.'
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
#     stenc = np.ones(orderStenc + 1)
    
    print('stenc:')
    print(stenc)
    degFreed = omega.degFreed
    hs = omega.h
    
    spots = np.roll(hs, -1) - hs
    
    print('off is', off)
    
    if (all(spots == 0)):
        p = []
        q = []
        NU = False
    else:
        # Index before fine-coarse interface
        p = np.where(spots > 0)[0][0]
        # Index before coarse-fine interface
        q = np.where(spots < 0)[0][0]
        NU = True
    
    print('orderStenc is', orderStenc)
    
    polyStencSet = [[] for i in range(orderStenc)]
    cellFaces = np.linspace(loBound, hiBound, num = orderStenc + 1)
    zeroLoc = np.where(cellFaces == 0)[0][0]
    print('cellFaces before:')
    print(cellFaces)
    print(np.where(cellFaces == 0)[0])
    cellFaces = np.delete(cellFaces, zeroLoc)
    print('cellFaces after:')
    print(cellFaces)
    

    for i in range(orderStenc):
        polyStencSet[i], n_c, n_f = GTT.CentGhost(omega, order, cellFaces[i])
    
    polyStencSet = np.asarray(polyStencSet)

    IMat = np.eye(degFreed, degFreed)
    
    # YOU'RE GONNA NEED THESE TO RESTRICT FOR HIGHER EVEN ORDERS, TOO.
    
    
    polyMatU = IMat + 0
    
    
    mat = np.zeros((degFreed, degFreed), float)
    derivOp = mat + 0
    
    for d in range(orderStenc + 1):
        s = int(off - d)
        
        derivMat = mat + 0
        np.fill_diagonal(derivMat, stenc[d])
        derivMat = np.roll(derivMat, s, axis = 0)
        
        polyMat = IMat + 0
        
        if (NU):
            if (s > 0):
                j = int(off - s)
                pAt = p
                pLow = (p - 1) % degFreed
                qAt = (q - s + 1) % degFreed #(q + 1) % degFreed
                for i in range (s):
                    polyMat[pAt, :] = 0
                    polyMat[pAt, pLow:pLow+2] = 0.5
                    polyMat[qAt, :] = polyStencSet[j, :]
                    pAt = (pAt - 1) % degFreed
                    pLow = (pLow - 2) % degFreed
                    qAt = (qAt + 1) % degFreed
                    j = int(j + 1)

            if (s < 0):
                j = int(off) # - s - 1
                qAt = (q + 1) % degFreed
                qLow = (q + 1) % degFreed
                pAt = (p + 1) % degFreed#p
                for i in range(abs(s)):
                    polyMat[qAt, :] = 0
                    polyMat[qAt, qLow:qLow+2] = 0.5
                    polyMat[pAt, :] = polyStencSet[j, :]
                    qAt = (qAt + 1) % degFreed
                    qLow = (qLow + 2) % degFreed
                    pAt = (pAt + 1) % degFreed
                    j = int(j + 1) # - 1
        
        matThis = derivMat @ polyMat
        
        derivOp = derivOp + matThis
    
#     derivOp = hMat @ derivOp

    rowSums = np.round(sum(derivOp, axis = 1), 11)
    probRows = np.where(rowSums != 0)[0]
    
    for row in probRows:
        for j in range(degFreed):
            frac = Fraction(derivOp[row, j]).limit_denominator(10**order)
            num = frac.numerator
            denom = frac.denominator
            derivOp[row, j] = num / denom
        
    return derivOp

In [12]:
if (((diff == 'U') or (diff == 'UD')) and (RL == 'L')):
    derivDiff = 'DD'
else:
    derivDiff = diff

deriv = SpaceDeriv(omegaAMR, order, derivDiff)#, RL, Ng)
print('D =')
print(deriv)
print('')

stenc:
[-0.01666666666666667  0.15000000000000002 -0.7500000000000001   0.                   0.7500000000000001  -0.15000000000000002  0.01666666666666667]
off is 3
orderStenc is 6
cellFaces before:
[-1.5 -1.  -0.5  0.   0.5  1.   1.5]
[3]
cellFaces after:
[-1.5 -1.  -0.5  0.5  1.   1.5]
D =
[[ 0.                      0.7500000000000001     -0.15000000000000002     0.01666666666666667     0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.      

In [13]:
if (RL == 'R'):
    face1  = faceOp[1:, :]
    face2 = faceOpOther[1:, :]
else:
    face1 = faceOpOther[:-1, :]
    face2 = faceOp[:-1, :]
    
derivShould = face1 - face2

In [14]:
print(deriv)

[[ 0.                      0.7500000000000001     -0.15000000000000002     0.01666666666666667     0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.           

In [15]:
print(derivShould)

[[ 0.                      0.75                   -0.15                    0.016666666666666666    0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.           

In [16]:
print(face1)

[[ 0.6166666666666667      0.6166666666666667     -0.13333333333333333     0.016666666666666666    0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.           

In [17]:
print(face2)

[[ 0.6166666666666667     -0.13333333333333333     0.016666666666666666    0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.                      0.           

In [18]:
difference = np.round(deriv - derivShould, 13)
print(difference)

[[ 0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0. -0.]
 [-0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.]
 [ 0. -0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.]
 [-0.  0. -0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  

In [19]:
print(np.count_nonzero(difference))

0


In [20]:
A = np.arange(10)
print(A)
print(A[2:])
print(A[-2:])
print(A[:2])
print(A[:-2])

[0 1 2 3 4 5 6 7 8 9]
[2 3 4 5 6 7 8 9]
[8 9]
[0 1]
[0 1 2 3 4 5 6 7]


In [21]:
print(omegaAMR.dx)
print(omegaAMR.h)
print('')

[0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.01171875 0.0078125  0.0078125  0.0078125  0.0078125  0.0078125  0.0078125  0.0078125  0.0078125  0.0078125  0.0078125  0.0078125  0.0078125  0.0078125  0.0078125  0.0078125  0.01171875 0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625   0.015625  ]
[0.015625  0.015625  0.015625  0.015625  0.015625  0.015625  0.015625  0.015625  0.015625  0.015625  0.015625  0.015625  0.015625  0.015625  0.015625  0.015625  0.015625  0.015625  0.015625  0.015625  0.015625  0.0156